# **DOWNLOAD SW REQUIREMENTS**

In [165]:
!pip install -q langchain==0.3.27 \
                langgraph==0.6.7 \
                langchain-openai==0.3.32 \
                langchain_experimental==0.3.4 \
                langchain_community==0.3.29 \
                python-dotenv==1.0.1 \
                langchain-core==0.3.75 \
                serpapi==0.1.5 \
                google-search-results==2.4.2 \
                exa_py==1.7.0

In [ ]:
!pip install yahooquery

# **IMPORTS · LIBRARIES · APIs**

In [166]:
import sys
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [167]:
%cd /content/drive/MyDrive/AIFI

/content/drive/MyDrive/AIFI


In [ ]:
from pydantic import BaseModel
from typing import TypedDict, Annotated, Literal, List, Dict, Any, Optional
from datetime import datetime, timedelta, timezone
import re
import time

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import os
import operator
import functools
from dotenv import load_dotenv

In [170]:
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_community.utilities import SerpAPIWrapper
from langchain_experimental.utilities import PythonREPL

In [ ]:
from langgraph.graph import StateGraph, END, START

In [ ]:
from IPython.display import Image, display

In [174]:
from dataclasses import dataclass

In [175]:
load_dotenv()
TAVILY_OPEN_KEY = os.getenv("TAVILY_OPEN_KEY")
tavily_tool = TavilySearchResults(max_results=8, search_depth="advanced", tavily_api_key=TAVILY_OPEN_KEY)
MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
llm = ChatOpenAI(model=MODEL, temperature=0.1)

# **CREATE AGENT**

In [ ]:
def create_agent(
    llm: ChatOpenAI,
    tools: list,
    system_prompt: str,
) -> str:
    """Create a function-calling agent and add it to the graph."""
    system_prompt += "\nWork autonomously according to your specialty, using the tools available to you."
    " Do not ask for clarification."
    " Your other team members (and other teams) will collaborate with you with their own specialties."
    " You are chosen for a reason! You are one of the following team members: {team_members}."
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_functions_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True)
    return executor

In [ ]:
def agent_node (state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [ ]:
def create_team_supervisor (llm: ChatOpenAI, system_prompt, members) -> str:
    """An LLM-based router"""
    options = ["FINISH"] + members
    function_def = {
        "name": "route",
        "description": "Select the next role.",
        "parameters": {
            "title": "routeSchema",
            "type": "object",
            "properties": {
                "next": {
                    "title": "Next",
                    "anyOf": [
                        {"enum": options},
                    ],
                },
            },
            "required": ["next"],
        },
    }

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            (
                "system",
                "Given the conversation above"
            ),
        ]
    ).partial(options=str(options), team_members=",".join(members))
    return(
        prompt
        | llm.bind_functions(functions=[function_def], function_call="route")
        | JsonOutputFunctionsParser()
    )

# **CLASS DEFINITIONS**

**Research Analyst, Risk Analyst and CIO agents follow the next Class**

In [ ]:
class FinanceTeamState (TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    team_members: str
    next: str
    current_files: str

In [ ]:
# def prelude (state):

# **UTILITIES**

## **PREPROCESSING**

In [178]:
def _log(state: dict, msg: str) -> dict:
    """Append logs sin acoplar a tipos del grafo."""
    logs = list(state.get("logs") or [])
    logs.append(msg)
    state["logs"] = logs
    return state


def _ensure_single_symbol_df(df: pd.DataFrame, ticker: str) -> pd.DataFrame:
    """Normalize yfinance frames that sometimes come with MultiIndex columns."""
    if isinstance(df.columns, pd.MultiIndex):
        try:
            df = df.xs(ticker, axis=1, level=1)
        except Exception:
            try:
                df = df.droplevel(1, axis=1)
            except Exception:
                pass
    return df


def _is_recent_iso8601(ts_str: str, days: int) -> bool:
    if not ts_str:
        return True
    try:
        ts = datetime.fromisoformat(ts_str.replace("Z",""))
        return (datetime.now(timezone.utc) - ts) <= timedelta(days=days)
    except Exception:
        return True

def _clean_text(text: str, max_len: int) -> str:
  text = re.sub(r'\s+', ' ', text.strip())  # Normalize whitespace (uses existing `import re`)
  return text[:max_len] + "..." if len(text) > max_len else text


## **INDICATORS HELPERS**

Takes a pandas DataFrame with historical share price data (OHLCV: Open, High, Low, Close, Volume) and calculates a series of standard technical indicators used in financial analysis.

- asof: Date of the last valid data in the DataFrame, formatted as ‘YYYY-MM-DD’.
- close: Adjusted closing price for the last available day.
- sma_10: Simple average of closing prices over the last 10 days.
- sma_20: Simple average of closing prices over the last 20 days.
- rsi_14: It measures the speed and change in price movements to detect overbought (>70) or oversold (<30) conditions.
- macd: Difference between EMA_12 and EMA_26, measuring momentum.
- macd_signal: 9-period EMA of the MACD.
- macd_hist: Difference between MACD and its signal.
- ret_1d: Percentage change in the closing price on the last day.
- ret_5d: Percentage change in closing price over the last 5 days.

In [ ]:
def compute_indicators_df(df: pd.DataFrame) -> Dict[str, Any]:
    df = _ensure_single_symbol_df(df, "_")
    close = df["Close"].astype(float)

    # SMA
    df["sma_10"] = close.rolling(10).mean()
    df["sma_20"] = close.rolling(20).mean()

    # RSI
    delta = close.diff()
    gain = delta.clip(lower=0)
    loss = (-delta).clip(lower=0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / (avg_loss.replace(0, np.nan))
    df["rsi_14"] = 100 - (100 / (1 + rs))

    # MACD
    ema12 = close.ewm(span=12, adjust=False).mean()
    ema26 = close.ewm(span=26, adjust=False).mean()
    macd = ema12 - ema26
    signal = macd.ewm(span=9, adjust=False).mean()
    hist = macd - signal
    df["macd"] = macd
    df["macd_signal"] = signal
    df["macd_hist"] = hist

    # simple returns
    df["ret_1d"] = close.pct_change(1)
    df["ret_5d"] = close.pct_change(5)

    last = df.dropna().iloc[-1]

    def _scalar(x):
        try:
            return float(x.iloc[0])
        except Exception:
            return float(x)

    return {
        "asof": last.name.strftime("%Y-%m-%d"),
        "close": _scalar(last["Close"]),
        "sma_10": _scalar(last["sma_10"]),
        "sma_20": _scalar(last["sma_20"]),
        "rsi_14": _scalar(last["rsi_14"]),
        "macd": _scalar(last["macd"]),
        "macd_signal": _scalar(last["macd_signal"]),
        "macd_hist": _scalar(last["macd_hist"]),
        "ret_1d": _scalar(last["ret_1d"]),
        "ret_5d": _scalar(last["ret_5d"]),
    }

Compose Indicator Queries generate a list of customised, indicator-aware search queries to find relevant news about a specific action.

Based topics:
- Earnings guidance.
- Product launches.
- Regulatory investigations, fines, or lawsuits.
- Acquisitions, mergers, or divestitures.
- Buybacks, dividends, or capital returns.
- Analyst updates.

Conditional based on indicators:
- RSI_14. If > 14, consider concerns about overbuying and profit-taking. If < 30, consider oversold rebound and positive catalysts.
- MACD_hist. If > 0, estimate positive momentum, exceeding estimates. If < 0, consider estimation failures, cuts and headwinds.
- Recent returns. If ret_5d > 0.03, positive 5-day return, consider why the stock rose last week. If ret_1d < -0.02 (significant daily decline), add the query asking about causes of downgrades or lawsuits.

In [ ]:
def compose_indicator_queries (ticker: str, ind: Dict[str, Any], days:int) -> List[str]:
  t = ticker.upper().strip()
  rsi = float(ind.get("rsi_14", 50) or 50)
  macd_hist = float(ind.get("macd_hist", 50) or 50)
  ret_1d = float(ind.get("ret_1d", 50) or 50)
  ret_5d = float(ind.get("ret_5d", 50) or 50)

  base_topics = [
      f"{t} earnings guidance last {days} days",
      f"{t} press release product launch last {days} days",
      f"{t} regulatory investigation fine lawsuit last {days} days",
      f"{t} acquisition merger divestidure last {days} days",
      f"{t} buyback dividend capital return last {days} days",
      f"{t} analyst upgrade downgrade last {days} days",
  ]

  # Settings for rsi.
  if rsi >= 70:
    base_topics += [f"{t} overbought profit taking valuation concerns last {days} days"]
  elif rsi <= 30:
    base_topics += [f"{t} oversold rebound catalyst upgrade last {days} days"]

  # Settings for MACD
  if macd_hist > 0:
    base_topics += [f"{t} momentum beat estimates guidance raise last {days} days"]
  elif macd_hist < 0:
    base_topics += [f"{t} miss estimates guidance cut headwinds last {days} days"]

  # Settings for ret
  if ret_5d >= 0.03:
    base_topics += [f"{t} why stock up last week investor reaction last {days} days"]
  if ret_1d <= -0.02:
    base_topics += [f"{t} why stock down today downgrade lawsuit last {days} days"]

  seen = set()
  queries = []
  for q in base_topics:
    if q not in seen:
      seen.add(q)
      queries.append(q)
  return queries[:10]

## **INDICATORS: AGENTS HELPERS**

INTERNAL USE:
- Class GraphState

In [ ]:
class GraphState(TypedDict):
    ticker: str
    _prices: Optional[pd.DataFrame]          
    indicators: Dict[str, Any]
    news: List[Dict[str, str]]               
    headlines: List[str]
    financials: Dict[str, Any]
    draft_md: str
    report: str
    logs: List[str]
    _cfg: Dict[str, Any]

In [ ]:
@tool("research_analyst")
def research_analyst(ticker: str, period: int):
    """1st - Research Analyst"""
    
    def validate_ticker(state: GraphState) -> GraphState:
        t = (state["ticker"] or "").upper().strip()
        if not t:
            raise ValueError("Empty ticker. Please provide a valid symbol (e.g., AAPL).")
        _log(state, f"[validate_ticker] Ticker => {t}")
        state["ticker"] = t
        return state
    
    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=1, max=8))
    def _dl_prices(ticker: str, days: int) -> pd.DataFrame:
        end = datetime.today()
        start = end - timedelta(days=days)
        df = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=False)
        if df is None or df.empty:
            raise RuntimeError("Prices could not be downloaded.")
        return df

    def load_prices(state: GraphState) -> GraphState:
        cfg = state["_cfg"]
        t = state["ticker"]
        df = _dl_prices(t, days=cfg["PRICE_LOOKBACK_DAYS"])
        df = _ensure_single_symbol_df(df, t)
        _log(state, f"[load_prices] OHLCV={list(df.columns)} rows={len(df)}")
        state["_prices"] = df
        return state

    def compute_indicators(state: GraphState) -> GraphState:
        df = state.get("_prices")
        if df is None or df.empty:
            raise RuntimeError("There are no prices in the state. Run “load_prices” first.")
        inds = compute_indicators_df(df)
        _log(state, f"[compute_indicators] keys={list(inds.keys())}")
        state["indicators"] = inds
        return state

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=0.5, max=6))
    def _tavily_run(query: str):
        try:
            return tavily_tool.run({"query": query})
        except TypeError:
            return tavily_tool.run(query)

    def _search_news_tavily_indicator_aware(ticker: str, indicators: Dict[str, Any],
                                            days: int, max_articles: int) -> List[NewsDoc]:
        queries = compose_indicator_queries(ticker, indicators, days=days)
        results, seen_urls = [], set()

        for q in queries:
            time.sleep(0.25)  # smooth rate-limit
            items = _tavily_run(q) or []
            for it in items:
                url = it.get("url") or it.get("source") or ""
                if not url or url in seen_urls:
                    continue
                title = it.get("title") or (it.get("content","").split("\n",1)[0][:120] if it.get("content") else "")
                excerpt = it.get("content") or it.get("snippet") or ""
                ts = it.get("published_date") or it.get("date") or ""

                if _is_recent_iso8601(ts, days=days):
                    results.append({
                        "title": _clean_text(title, 200),
                        "url": url,
                        "excerpt": _clean_text(excerpt, 900),
                        "ts": ts
                    })
                    seen_urls.add(url)
            if len(seen_urls) >= max_articles:
                break

        # sort by date descending (no date at the end)
        def _key(x):
            ts = x.get("ts","")
            try:
                return datetime.fromisoformat(ts.replace("Z",""))
            except Exception:
                return datetime.min
        results_sorted = sorted(results, key=_key, reverse=True)[:max_articles]

        return [NewsDoc(title=r["title"], url=r["url"], excerpt=r["excerpt"]) for r in results_sorted]

    # --- NODE: news ---
    def gather_news(state: GraphState) -> GraphState:
        cfg = state["_cfg"]
        t = state["ticker"]
        ind = state.get("indicators") or {}
        news_docs = _search_news_tavily_indicator_aware(
            t, ind, days=cfg["DAYS_NEWS"], max_articles=cfg["MAX_NEWS"]
        )
        headlines = [nd.title for nd in news_docs[:cfg["HEADLINES_TOPK"]]]
        _log(state, f"[gather_news] found={len(news_docs)}")
        state["news"] = [nd.model_dump() for nd in news_docs]
        state["headlines"] = headlines
        return state

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=1, max=8))
    def _fetch_financials(ticker: str) -> Dict[str, Any]:
        t = yf.Ticker(ticker)
        info = {}
        try:
            info = t.get_info()
        except Exception:
            info = {}
        fast = getattr(t, "fast_info", {}) or {}

        def _get(*keys, default=None):
            for k in keys:
                if k in fast: return fast[k]
                if k in info: return info[k]
            return default

        return {
            "symbol": ticker,
            "longName": info.get("longName") or info.get("shortName") or ticker,
            "currency": _get("currency"),
            "price": float(_get("last_price","lastPrice","regularMarketPrice", default=np.nan)),
            "market_cap": _get("marketCap"),
            "trailingPE": _get("trailingPE"),
            "forwardPE": _get("forwardPE"),
            "volume": _get("last_volume","volume"),
            "52w_high": _get("year_high","fiftyTwoWeekHigh"),
            "52w_low": _get("year_low","fiftyTwoWeekLow"),
            "asof": datetime.now(timezone.utc).isoformat() + "Z",
        }

    def get_financials(state: GraphState) -> GraphState:
        t = state["ticker"]
        fin = _fetch_financials(t)
        _log(state, f"[get_financials] price={fin.get('price')} market_cap={fin.get('market_cap')}")
        state["financials"] = fin
        return state

    def _format_news_bullets(news: List[Dict[str,str]], k: int) -> str:
        items = news[:k] if news else []
        lines = []
        for it in items:
            title = it.get("title","").strip()
            url = it.get("url","").strip()
            if title and url:
                lines.append(f"- {title} ({url})")
        return "\n\n".join(lines)

    def draft_report(state: GraphState) -> GraphState:
        cfg = state["_cfg"]
        t = state["ticker"]
        fin = state.get("financials") or {}
        news = state.get("news") or []
        inds = state.get("indicators") or {}
        news_section = _format_news_bullets(news, k=5)

        prompt = ChatPromptTemplate.from_messages([
            ("system",
            "You are a concise, factual financial analyst. Write in clear bullets when useful. "
            "Cite sources in 'Recent Developments' as the provided URLs. Avoid fabricating data."),
            ("human",
            """Create a report with EXACTLY these sections:

    1) Executive Summary
    2) Current Valuation
    3) Recent Developments (3–5 bullets, include provided URLs)
    4) Analyst Perspective

    Ticker: {ticker}

    Facts you MUST use:
    - Financial snapshot: {financials}
    - Indicators (context only, do not overfit): {indicators}
    - Curated news (use for section 3):
    {news_bullets}

    Guidelines:
    - Be specific and avoid hype.
    - In 'Current Valuation', show price (with currency if known), market cap and P/E (trailing/forward when available).
    - In 'Analyst Perspective', discuss upside/downside catalysts and risks.
    """)
        ])

        msg = prompt.format_messages(
            ticker=t,
            financials=fin,
            indicators=inds,
            news_bullets=news_section
        )
        out = llm.invoke(msg).content
        _log(state, "[draft_report] completed")
        state["report"] = out
        return state


In [ ]:
@tool("risk_analyst")
def risk_analyst (text: str) -> str:

    def sentiment_analysis (text:str) -> str:
        from transformers import AutoTokenizer, AutoModelForSequenceClassification
        import torch

        tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
        model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        sentiment = torch.argmax(predictions, dim=1).item()

        labels = {0: "Negative", 1: "Neutral", 2: "Positive"}
        return f"Sentiment: {labels[sentiment]}, Confidence: {predictions[0][sentiment].item():.2f}"




# **INVOKE AGENTS**

In [ ]:
research_analyst_agent = create_agent(
    llm,
    [research_analyst],
    """"""
)
research_node = functools.partial(agent_node, agent=research_analyst_agent, name="ResearchAnalyst")

risk_analyst_agent = create_agent(
    llm,
    [risk_analyst],
    """"""
)
risk_node = functools.partial(agent_node, agent=risk_analyst_agent, name="RiskAnalyst")

cio = create_team_supervisor(
    llm,
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {team_members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.",
    ["ResearchAnalyst","RiskAnalyst"],
)

# **GRAPH**

In [ ]:
# Create the Graph
financial_graph = StateGraph(FinanceTeamState)
financial_graph.add_note("ResearchAnalyst", research_analyst)
financial_graph.add_note("RiskAnalyst", risk_analyst)
financial_graph.add_note("cio", cio)

# Add the Edges
financial_graph.add_edge("ResearchAnalyst", "cio")
financial_graph.add_edge("RiskAnalyst", "cio")

# Add the Edges where Routing Applies
financial_graph.add_conditional_edges(
    "cio",
    lambda x: x["next"],
    {
        "ResearchAnalyst": "Research Analyst",
        "RiskAnalyst": "RiskAnalyst",
        "FINISH": END,
    }
)
financial_graph.add_edge(START, "cio")
chain = financial_graph.compile()

In [ ]:
def enter_chain(message: str, members: List[str]):
    results = {
        "messages": [HumanMessage(content=message)],
        "team_members": ", ".join(members),
    }
    return results

In [ ]:
financial_chain = (
    functools.partial(enter_chain, members=financial_graph.nodes)
    | financial_graph.compile()
)

# APP

In [ ]:
@dataclass
class Config:
    DAYS_NEWS: int = 90           # "last quarter"
    MAX_NEWS: int = 8
    HEADLINES_TOPK: int = 5
    PRICE_LOOKBACK_DAYS: int = 180

def run_multiagent(ticker: str, cfg: Optional[Config] = None) -> GraphState:
    cfg = cfg or Config()
    app = build_graph()
    init: GraphState = {
        "ticker": ticker,
        "_prices": None,
        "indicators": {},
        "news": [],
        "headlines": [],
        "financials": {},
        "report": "",
        "logs": [],
        "_cfg": {
            "DAYS_NEWS": cfg.DAYS_NEWS,
            "MAX_NEWS": cfg.MAX_NEWS,
            "HEADLINES_TOPK": cfg.HEADLINES_TOPK,
            "PRICE_LOOKBACK_DAYS": cfg.PRICE_LOOKBACK_DAYS
        },
    }
    final_state: GraphState = app.invoke(init)
    return final_state

# **USER**

The user can set the following parameters:

1. DAYS_NEWS: number of days with news.
2. MAX_NEWS: a limit of reported news
3. HEADLINES_TOPK: maximum number of items you intend to bring.
4. PRICE_LOOKBACK_DAYS: number of days back that the system downloads historical prices

In [184]:
from rich.console import Console
from rich.panel import Panel
from rich.table import Table

In [185]:
console = Console()

In [186]:
# =======================
# EXECUTION CELL (robust)
# =======================
from pprint import pprint

# --- Config by default (could be adapted to your interests) ---
try:
    # Usa la clase Config definida anteriormente
    cfg = Config(
        DAYS_NEWS=30,
        MAX_NEWS=8,
        HEADLINES_TOPK=5,
        PRICE_LOOKBACK_DAYS=180
    )
except NameError:
    # Fallback if there's no configuration.
    cfg = type("Config", (), {
        "DAYS_NEWS": 30,
        "MAX_NEWS": 8,
        "HEADLINES_TOPK": 5,
        "PRICE_LOOKBACK_DAYS": 180
    })()

TICKERS = ["XYL", "AAPL", "MSFT", "GOOGL"]  # <-- change or add more tickets here.

def _execute_sequential(ticker: str, cfg_obj) -> dict:
    """Fallback secuencial por si el grafo da error con '_cfg' u otro."""
    init_state = {
        "ticker": ticker,
        "_prices": None,
        "indicators": {},
        "news": [],
        "headlines": [],
        "financials": {},
        "report": "",
        "logs": [],
        "_cfg": {
            "DAYS_NEWS": cfg_obj.DAYS_NEWS,
            "MAX_NEWS": cfg_obj.MAX_NEWS,
            "HEADLINES_TOPK": cfg_obj.HEADLINES_TOPK,
            "PRICE_LOOKBACK_DAYS": cfg_obj.PRICE_LOOKBACK_DAYS
        },
    }
    # Nodes following order established.
    state = init_state
    for fn in [validate_ticker, load_prices, compute_indicators, gather_news, get_financials, draft_report]:
        state = fn(state)
    return state

for tk in TICKERS:
    #print("=" * 100)
    #print(f"Running agent for: {tk}")
    console.rule(f"[bold]Running agent for: {tk}")

    state = None
    # 1) Try with run_agent:
    if "run_agent" in globals():
        try:
            state = run_agent(tk, cfg)
        except KeyError as e:
            if str(e) == "'_cfg'":
                print("[WARN] KeyError '_cfg' in graph. Running sequential fallback...")
                state = _execute_sequential(tk, cfg)
            else:
                raise
        except Exception as e:
            print(f"[WARN] Error in graph: {e}\nRetrying with sequential execution...")
            state = _execute_sequential(tk, cfg)
    else:
        # 2) If there's no a run_agent, use a sequential mode directly:
        state = _execute_sequential(tk, cfg)

    # --- Output ---
    #print("\n--- Current Valuation (raw) ---")
    #pprint(state.get("financials"))

    #print("\n--- Recent Headlines ---")
    #for h in state.get("headlines", []):
    #    print(f"- {h}")
    console.rule("[bold green]Final Report")
    console.print(Panel.fit(state.get("report", "").strip()))


───────────────────────────────────────────── Running agent for: XYL ──────────────────────────────────────────────

────────────────────────────────────────────────── Final Report ───────────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ # Xylem Inc. (XYL) Financial Report                                                                             │
│                                                                                                                 │
│ ## 1) Executive Summary                                                                                         │
│ Xylem Inc. (NYSE: XYL) is a leading global water technology company focused on developing innovative solutions  │
│ for water and wastewater management. As of September 22, 2025, Xylem's stock is trading at approximately        │
│ $143.54, reflecting a market capitalization of about $34.94 billion. The company operates in a sector that is   │
│ increasingly critical due to global water scarcity and infrastructure needs.                                    │
│                                                                                                                 │
│ ## 2) Current Valuation                                                                                         │
│ - **Price**: $143.54 USD                                                                                        │
│ - **Market Cap**: $34.94 billion USD                                                                            │
│ - **Trailing P/E**: 37.38                                                                                       │
│ - **Forward P/E**: 30.22                                                                                        │
│ - **52-Week High**: $145.33                                                                                     │
│ - **52-Week Low**: $100.47                                                                                      │
│ - **Volume**: 933,182 shares traded                                                                             │
│                                                                                                                 │
│ ## 3) Recent Developments                                                                                       │
│ - Xylem is scheduled to report its earnings on October 30, 2025, as indicated by Zacks                          │
│ (https://www.zacks.com/stock/research/XYL/earnings-calendar).                                                   │
│ - Mn Services Vermogensbeheer B.V. has acquired $11.78 million worth of Xylem stock, suggesting institutional   │
│ interest in the company                                                                                         │
│ (https://www.marketbeat.com/instant-alerts/filing-xylem-inc-xyl-shares-bought-by-mn-services-vermogensbeheer-bv │
│ -2025-09-20/).                                                                                                  │
│ - The company has been actively updating its investors through press releases, highlighting ongoing projects    │
│ and financial performance (https://xyleminc.gcs-web.com/news-events/news-releases).                             │
│ - Detailed earnings estimates for Xylem indicate a positive outlook for the upcoming quarter                    │
│ (https://www.zacks.com/stock/quote/XYL/detailed-earning-estimates).                                             │
│ - MarketBeat provides insights into Xylem's earnings date and reports, reinforcing the company's transparency   │
│ with investors (https://www.marketbeat.com/stocks/NYSE/XYL/earnings/).                                          │
│                                                                                                                 │
│ ## 4) Analyst Perspective                                                                                       │
│ ### Upside Catalysts:                                                                                           │
│ - **Growing Demand**: Increasing global focus on water

───────────────────────────────────────────── Running agent for: AAPL ─────────────────────────────────────────────

────────────────────────────────────────────────── Final Report ───────────────────────────────────────────────────

...

────────────────────────────────────────────────── Final Report ───────────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ # Microsoft Corporation (MSFT) Financial Report                                                                 │
│                                                                                                                 │
│ ## 1) Executive Summary                                                                                         │
│ Microsoft Corporation (MSFT) is a leading technology company known for its software products, cloud services,   │
│ and hardware. As of September 22, 2025, MSFT is trading at approximately $514.45, with a market capitalization  │
│ of about $3.82 trillion. The company has shown resilience in its financial performance, supported by strong     │
│ demand for its cloud services and software solutions.                                                           │
│                                                                                                                 │
│ ## 2) Current Valuation                                                                                         │
│ - **Price:** $514.45 USD                                                                                        │
│ - **Market Capitalization:** $3.82 trillion                                                                     │
│ - **Trailing P/E Ratio:** 37.77                                                                                 │
│ - **Forward P/E Ratio:** 34.41                                                                                  │
│ - **52-Week High:** $555.45                                                                                     │
│ - **52-Week Low:** $344.79                                                                                      │
│ - **Volume:** 19,828,005 shares                                                                                 │
│                                                                                                                 │
│ ## 3) Recent Developments                                                                                       │
│ - Microsoft is scheduled to report its FY25 Q4 earnings, with expectations of continued growth in cloud         │
│ services and software sales. [MarketBeat](https://www.marketbeat.com/stocks/NASDAQ/MSFT/earnings/)              │
│ - The company has released a press statement regarding its FY25 Q4 earnings, highlighting key performance       │
│ metrics and future outlook. [Microsoft Investor                                                                 │
│ Relations](https://www.microsoft.com/en-us/investor/earnings/fy-2025-q4/press-release-webcast)                  │
│ - Upcoming earnings dates for MSFT have been confirmed, indicating a consistent schedule for financial          │
│ reporting. [Market Chameleon](https://marketchameleon.com/Overview/MSFT/Earnings/Earnings-Dates/)               │
│ - Analysts are closely monitoring MSFT's earnings report date, which is anticipated to provide insights into    │
│ the company's performance amid a competitive market.                                                            │
│ [NASDAQ](https://www.nasdaq.com/market-activity/stocks/msft/earnings)                                           │
│ - Recent earnings dates and reports have shown a trend of strong revenue growth, particularly in the cloud      │
│ computing segment. [Investing.com](https://www.investing.com/equities/microsoft-corp-earnings)                  │
│                                                                                                                 │
│ ## 4) Analyst Perspective                                                                                       │
│ - **Upside Catalysts:**                                                                                         │
│   - Continued growth in cloud services (Azure) is expe

──────────────────────────────────────────── Running agent for: GOOGL ─────────────────────────────────────────────

────────────────────────────────────────────────── Final Report ───────────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ # Alphabet Inc. (GOOGL) Financial Report                                                                        │
│                                                                                                                 │
│ ## 1) Executive Summary                                                                                         │
│ Alphabet Inc. (GOOGL) is a leading technology company primarily known for its search engine, digital            │
│ advertising, and cloud computing services. As of September 22, 2025, GOOGL's stock price is $252.53, reflecting │
│ a robust market capitalization of approximately $3.02 trillion. The company is positioned for growth, driven by │
│ advancements in cloud computing and artificial intelligence.                                                    │
│                                                                                                                 │
│ ## 2) Current Valuation                                                                                         │
│ - **Price**: $252.53 USD                                                                                        │
│ - **Market Capitalization**: $3.02 trillion                                                                     │
│ - **Trailing P/E Ratio**: 26.95                                                                                 │
│ - **Forward P/E Ratio**: 28.18                                                                                  │
│ - **Volume**: 31,976,483 shares                                                                                 │
│ - **52-Week High**: $256.00                                                                                     │
│ - **52-Week Low**: $140.53                                                                                      │
│                                                                                                                 │
│ ## 3) Recent Developments                                                                                       │
│ - Analysts are optimistic about GOOGL's growth potential, particularly in cloud services and AI, as highlighted │
│ in the earnings preview (source:                                                                                │
│ [MarketPulse](https://www.marketpulse.com/markets/alphabet-googl-earnings-preview-cloud-and-ai-drive-growth/)). │
│ - Upcoming earnings reports are anticipated to provide insights into the company's performance and strategic    │
│ direction (source: [MarketBeat](https://www.marketbeat.com/stocks/NASDAQ/GOOGL/earnings/)).                     │
│ - Analyst ratings indicate a generally positive outlook for GOOGL, with many expecting continued revenue growth │
│ (source: [Yahoo Finance - GOOGL](https://finance.yahoo.com/quote/GOOGL/analysis/)).                             │
│ - The stock has shown strong momentum, with a recent RSI of 89.17 indicating overbought conditions (contextual  │
│ data).                                                                                                          │
│ - The MACD indicator suggests bullish momentum, with a recent histogram value of 0.99 (contextual data).        │
│                                                                                                                 │
│ ## 4) Analyst Perspective                                                                                       │
│ - **Upside Catalysts**:                                                                                         │
│   - Continued growth in cloud computing and AI services could significantly enhance revenue streams.            │
│   - Positive analyst sentiment and ratings may drive investor confidence and stock price appreciation.          │
│                                                       